**Search Agent**

In [1]:
morocco_map = {
    'Casablanca': ['Rabat', 'El Jadida'],
    'Rabat':      ['Casablanca', 'Meknes', 'Tangier'],
    'El Jadida':  ['Casablanca', 'Marrakech'],
    'Meknes':     ['Rabat', 'Fez', 'Ifrane'],
    'Tangier':    ['Rabat'],
    'Marrakech':  ['El Jadida'],
    'Fez':        ['Meknes', 'Ifrane'],
    'Ifrane':     ['Meknes', 'Fez']
}

**BFS**

In [ ]:
def bfs_agent(graph, start, goal):
    # A Queue to keep track of paths we need to explore
    # We start with just the starting city
    queue = [[start]]
    
    # A set to remember where we have already been
    # A set is a list that does not allow duplicates
    visited = set()

    while queue:
        # 1. Get the first path from the queue (First-In, First-Out)
        current_path = queue.pop(0)
        
        # 2. Get the last node in this path (where we are currently)
        current_city = current_path[-1]

        # 3. Check if we reached the goal
        if current_city == goal:
            return current_path
        
        # 4. If not, look at neighbors
        if current_city not in visited:
            visited.add(current_city)
            
            # Add new paths for all neighbors
            for neighbor in graph[current_city]:
                new_path = list(current_path)
                new_path.append(neighbor)
                queue.append(new_path)
                
    return "No path found"


print("BFS Route:", bfs_agent(morocco_map, 'Meknes', 'Casablanca'))

BFS Route: ['Meknes', 'Rabat', 'Casablanca']


**DFS**

In [5]:
def dfs_agent(graph, start, goal):
    # A Stack to keep track of paths (Last-In, First-Out)
    stack = [[start]]
    visited = set()

    while stack:
        # 1. Get the LAST path added to the stack
        current_path = stack.pop() 
        current_city = current_path[-1]

        if current_city == goal:
            return current_path

        if current_city not in visited:
            visited.add(current_city)
            
            # Add new paths for all neighbors
            for neighbor in graph[current_city]:
                new_path = list(current_path)
                new_path.append(neighbor)
                stack.append(new_path)

    return "No path found"


print("DFS Route:", dfs_agent(morocco_map, 'Meknes', 'Casablanca'))

DFS Route: ['Meknes', 'Rabat', 'Casablanca']


**Heuristics**

In [4]:
import math
import heapq  


morocco_map = {
    'Casablanca': ['Rabat', 'El Jadida'],
    'Rabat':      ['Casablanca', 'Meknes', 'Tangier'],
    'El Jadida':  ['Casablanca', 'Marrakech'],
    'Meknes':     ['Rabat', 'Fez', 'Ifrane'],
    'Tangier':    ['Rabat'],
    'Marrakech':  ['El Jadida'],
    'Fez':        ['Meknes', 'Ifrane'],
    'Ifrane':     ['Meknes', 'Fez']
}


# Tangier is North (High Y) and Marrakech is South (Low Y)
city_coords = {
    'Tangier':    (1, 10),
    'Rabat':      (0.5, 8),
    'Casablanca': (0, 6),
    'El Jadida':  (-0.5, 4),
    'Marrakech':  (-0.5, 0),
    'Meknes':     (2, 7),
    'Fez':        (3, 7.5),
    'Ifrane':     (2.5, 5)
}

**Euclidean and Manhattan distances**

In [1]:
def get_heuristic(city, goal, method='euclidean'):
    x1, y1 = city_coords[city]
    x2, y2 = city_coords[goal]
    
    if method == 'manhattan':
        # You can only move along grid lines (Up/Down/Left/Right)
        return abs(x1 - x2) + abs(y1 - y2)
        
    elif method == 'euclidean':
        # Diagonal movement allowed
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    
    else:
        return 0 # If heuristic is 0, A* turns back into BFS

**(A*)**

In [6]:
def a_star_agent(graph, start, goal, heuristic_type='euclidean'):
    # We store tuples: (Score, Current_City, Path_List)
    # The heap automatically sorts by the first item (Score)
    priority_queue = []
    
    # Calculate initial score (0 cost + estimated distance)
    initial_h = get_heuristic(start, goal, heuristic_type)
    heapq.heappush(priority_queue, (initial_h, start, [start]))
    
    # To keep track of the lowest cost to reach a node so far
    costs_so_far = {start: 0}

    print(f"Starting A* Search using {heuristic_type}")

    while priority_queue:
        # 1. Get the path with the LOWEST Score
        current_score, current_city, path = heapq.heappop(priority_queue)
        
        # 2. Check Goal
        if current_city == goal:
            return path
        
        # 3. Explore Neighbors
        for neighbor in graph[current_city]:
            # CALCULATE G: Cost to get here + 1 hop to neighbor
            new_cost = costs_so_far[current_city] + 1 
            
            # If we haven't visited this neighbor, OR we found a faster way to it
            if neighbor not in costs_so_far or new_cost < costs_so_far[neighbor]:
                costs_so_far[neighbor] = new_cost
                
                # CALCULATE H: Estimate from neighbor to goal
                heuristic = get_heuristic(neighbor, goal, heuristic_type)
                
                # CALCULATE F: Total Score
                final_score = new_cost + heuristic
                
                # Add to fringe
                new_path = list(path)
                new_path.append(neighbor)
                heapq.heappush(priority_queue, (final_score, neighbor, new_path))
                
                print(f"  Checked {neighbor}: Cost(g)={new_cost}, Guess(h)={heuristic:.1f}, Score(f)={final_score:.1f}")

    return "No path found"

In [7]:
print("A* Route (Euclidean):", a_star_agent(morocco_map, 'Meknes', 'Casablanca', 'euclidean'))
print("A* Route (Manhattan):", a_star_agent(morocco_map, 'Meknes', 'Casablanca', 'manhattan'))  

Starting A* Search using euclidean
  Checked Rabat: Cost(g)=1, Guess(h)=2.1, Score(f)=3.1
  Checked Fez: Cost(g)=1, Guess(h)=3.4, Score(f)=4.4
  Checked Ifrane: Cost(g)=1, Guess(h)=2.7, Score(f)=3.7
  Checked Casablanca: Cost(g)=2, Guess(h)=0.0, Score(f)=2.0
  Checked Tangier: Cost(g)=2, Guess(h)=4.1, Score(f)=6.1
A* Route (Euclidean): ['Meknes', 'Rabat', 'Casablanca']
Starting A* Search using manhattan
  Checked Rabat: Cost(g)=1, Guess(h)=2.5, Score(f)=3.5
  Checked Fez: Cost(g)=1, Guess(h)=4.5, Score(f)=5.5
  Checked Ifrane: Cost(g)=1, Guess(h)=3.5, Score(f)=4.5
  Checked Casablanca: Cost(g)=2, Guess(h)=0.0, Score(f)=2.0
  Checked Tangier: Cost(g)=2, Guess(h)=5.0, Score(f)=7.0
A* Route (Manhattan): ['Meknes', 'Rabat', 'Casablanca']


**Min-Max**

In [8]:
# THE BOARD: A simple list of 9 items
board = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

# 1. SIMPLE WIN CHECKER
# Instead of complex loops, we just list the 8 ways to win
def check_winner(b, player):
    # Check Rows
    if b[0] == player and b[1] == player and b[2] == player: return True
    if b[3] == player and b[4] == player and b[5] == player: return True
    if b[6] == player and b[7] == player and b[8] == player: return True
    # Check Columns
    if b[0] == player and b[3] == player and b[6] == player: return True
    if b[1] == player and b[4] == player and b[7] == player: return True
    if b[2] == player and b[5] == player and b[8] == player: return True
    # Check Diagonals
    if b[0] == player and b[4] == player and b[8] == player: return True
    if b[2] == player and b[4] == player and b[6] == player: return True
    return False


def get_best_move(b):
    # RULE 1: Can I (AI 'O') win right now?
    for i in range(9):
        if b[i] == ' ':            # If spot is empty
            b[i] = 'O'             # Try moving there
            if check_winner(b, 'O'):
                return i           # FOUND A WINNING MOVE!
            b[i] = ' '             # Reset spot (Undo)

    # RULE 2: Will the Human ('X') win next?
    for i in range(9):
        if b[i] == ' ':
            b[i] = 'X'             # Pretend to be the Human
            if check_winner(b, 'X'):
                b[i] = 'O'         # Block them!
                return i
            b[i] = ' '             # Reset

    # RULE 3: Just take the center if open (It's the best spot)
    if b[4] == ' ':
        return 4

    # RULE 4: Pick the first open corner
    for i in [0, 2, 6, 8]:
        if b[i] == ' ':
            return i

    # RULE 5: Pick whatever is left
    for i in range(9):
        if b[i] == ' ':
            return i

# Main Game Loop
print("--- Simple Tic-Tac-Toe ---")
print("You are X. The AI is O.")
print(" 0 | 1 | 2 ")
print("---+---+---")
print(" 3 | 4 | 5 ")
print("---+---+---")
print(" 6 | 7 | 8 \n")

while True:
    # 1. Human Move
    move = int(input("Enter position (0-8): "))
    if board[move] != ' ':
        print("Occupied! Try again.")
        continue
    board[move] = 'X'

    # Check if Human Won
    if check_winner(board, 'X'):
        print("You Win!")
        break

    # Check for Tie (Board full)
    if ' ' not in board:
        print("It's a Tie!")
        break

    # 2. AI Move
    ai_move = get_best_move(board)
    board[ai_move] = 'O'
    
    # Print Board
    print(f"\nAI chose {ai_move}:")
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print(f" {board[6]} | {board[7]} | {board[8]} ")

    if check_winner(board, 'O'):
        print("AI Wins!")
        break

--- Simple Tic-Tac-Toe ---
You are X. The AI is O.
 0 | 1 | 2 
---+---+---
 3 | 4 | 5 
---+---+---
 6 | 7 | 8 


AI chose 4:
 X |   |   
   | O |   
   |   |   
Occupied! Try again.

AI chose 6:
 X |   |   
 X | O |   
 O |   |   

AI chose 1:
 X | O | X 
 X | O |   
 O |   |   

AI chose 8:
 X | O | X 
 X | O |   
 O | X | O 
It's a Tie!
